<a href="https://colab.research.google.com/github/frances-uy/s2025-assignment2-data/blob/master/Uy%2CFrances_Michelle_Assignment_2_Outputs_ECE491B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Quality Classifier for Text Content
## CS336 Assignment: Filtering Language Modeling Data

This notebook demonstrates a quality classifier that identifies high-quality vs. low-quality text content. It's designed to classify text as either from a high-quality source (like Wikipedia references) or a low-quality source (like random web crawl data).

### Setup

In [60]:
!git clone https://github.com/frances-uy/s2025-assignment2-data.git

Cloning into 's2025-assignment2-data'...
remote: Enumerating objects: 139, done.
remote: Counting objects: 100% (47/47), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 139 (delta 29), reused 32 (delta 22), pack-reused 92 (from 1)
Receiving objects: 100% (139/139), 16.93 MiB | 33.14 MiB/s, done.
Resolving deltas: 100% (42/42), done.


In [61]:
!ls -la

total 16632
drwxr-xr-x 6 root root     4096 Mar 29 03:51 .
drwxr-xr-x 6 root root     4096 Mar 29 03:05 ..
-rw-r--r-- 1 root root     1102 Mar 29 03:05 CHANGELOG.md
-rw-r--r-- 1 root root     3764 Mar 29 03:05 compare_extraction.py
-rw-r--r-- 1 root root    15518 Mar 29 03:05 create_training_data.py
drwxr-xr-x 5 root root     4096 Mar 29 03:05 cs336-basics
drwxr-xr-x 5 root root     4096 Mar 29 03:05 cs336-data
-rw-r--r-- 1 root root   149544 Mar 29 03:05 cs336_spring2024_assignment4_data.pdf
drwxr-xr-x 8 root root     4096 Mar 29 03:37 .git
-rw-r--r-- 1 root root     3112 Mar 29 03:05 .gitignore
-rw-r--r-- 1 root root  3934088 Mar 29 03:05 quality_train.txt
-rw-r--r-- 1 root root     4763 Mar 29 03:05 README.md
drwxr-xr-x 5 root root     4096 Mar 29 03:51 s2025-assignment2-data
-rwxr-xr-x 1 root root     1042 Mar 29 03:05 test_and_make_submission.sh
-rw-r--r-- 1 root root      991 Mar 29 03:05 train_model.py
-rw-r--r-- 1 root root     3837 Mar 29 03:05 train_quality_model.py
-rw-r--r-

In [ ]:
!pip install -e ./cs336-basics/
!pip install -e './cs336-data/[test]'

Obtaining file:///content/s2025-assignment2-data/s2025-assignment2-data/cs336-basics
  Preparing metadata (setup.py) ... done
  Attempting uninstall: cs336_basics
    Found existing installation: cs336_basics 0.0.0.dev0
    Uninstalling cs336_basics-0.0.0.dev0:
      Successfully uninstalled cs336_basics-0.0.0.dev0
  Running setup.py develop for cs336_basics
Obtaining file:///content/s2025-assignment2-data/s2025-assignment2-data/cs336-data
  Preparing metadata (setup.py) ... done
  Attempting uninstall: cs336_data
    Found existing installation: cs336_data 0.0.4
    Uninstalling cs336_data-0.0.4:
      Successfully uninstalled cs336_data-0.0.4
  Running setup.py develop for cs336_data


In [92]:
!git checkout master

Already on 'master'
Your branch is up to date with 'origin/master'.


In [93]:
!git pull

Already up to date.


In [79]:
!ls

CHANGELOG.md		 cs336_spring2024_assignment4_data.pdf	train_model.py
compare_extraction.py	 quality_train.txt			train_quality_model.py
create_training_data.py  README.md				wiki_sample.warc.gz
cs336-basics		 s2025-assignment2-data
cs336-data		 test_and_make_submission.sh


In [82]:
!pwd

/content/s2025-assignment2-data/s2025-assignment2-data


In [91]:
!git checkout master

Already on 'master'
Your branch is up to date with 'origin/master'.


In [72]:
from __future__ import annotations

import os
import re
from typing import Tuple, Any
import fasttext

## 2.2: Problem (extract_text): 3 points

In [71]:
from resiliparse.extract.html2text import extract_plain_text
from resiliparse.parse.encoding import detect_encoding

def extract_text_from_html_bytes(html_bytes):
    """
    Extract plain text from HTML byte string.

    Args:
        html_bytes (bytes): Raw HTML content as bytes

    Returns:
        str: Extracted plain text
    """
    # First try UTF-8 decoding
    try:
        html_str = html_bytes.decode('utf-8')
    except UnicodeDecodeError:
        # If UTF-8 fails, try to detect the encoding
        detected_encoding = detect_encoding(html_bytes)
        if detected_encoding:
            try:
                html_str = html_bytes.decode(detected_encoding)
            except UnicodeDecodeError:
                # If all else fails, use 'replace' to handle decoding errors
                html_str = html_bytes.decode('utf-8', errors='replace')
        else:
            # Fallback with error replacement
            html_str = html_bytes.decode('utf-8', errors='replace')

    # Extract plain text using Resiliparse
    extracted_text = extract_plain_text(html_str)
    return extracted_text

In [75]:
def run_extract_text_from_html_bytes(html_bytes: bytes) -> str | None:
    return extract_impl(html_bytes)

## Quality Classifier for Text Content (Section 2.7)

In [70]:
import fasttext
import re
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from typing import Tuple